In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

# Paths to training and testing sets
train_dir = '/Users/rohanrao/Desktop/hair4face/mens faceshape/training_set'
test_dir = '/Users/rohanrao/Desktop/hair4face/mens faceshape/testing_set'

# Image parameters
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

# Data generators (folders: oblong, oval, square, round)
train_gen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_data = test_gen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Model definition
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=IMG_SIZE + (3,)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(4, activation='softmax')  # 4 classes: oblong, oval, square, round
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_data, epochs=10, validation_data=test_data)

# Evaluate
loss, acc = model.evaluate(test_data)
print(f'Test accuracy: {acc:.2f}')

Found 927 images belonging to 4 classes.
Found 384 images belonging to 4 classes.


/Users/rohanrao/Desktop/hair4face/.venv312/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/rohanrao/Desktop/hair4face/.venv312/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 12s 392ms/step - accuracy: 0.2848 - loss: 2.0486 - val_accuracy: 0.1927 - val_loss: 1.4081
Epoch 2/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 6s 191ms/step - accuracy: 0.3765 - loss: 1.3436 - val_accuracy: 0.2188 - val_loss: 1.4385
Epoch 3/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 5s 190ms/step - accuracy: 0.4800 - loss: 1.2293 - val_accuracy: 0.2943 - val_loss: 1.4198
Epoch 4/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 5s 172ms/step - accuracy: 0.5944 - loss: 1.0006 - val_accuracy: 0.3438 - val_loss: 1.6018
Epoch 5/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 5s 169ms/step - accuracy: 0.7497 - loss: 0.6935 - val_accuracy: 0.2760 - val_loss: 1.6359
Epoch 6/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 5s 178ms/step - accuracy: 0.8781 - loss: 0.4079 - val_accuracy: 0.3021 - val_loss: 2.1098
Epoch 7/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 5s 156ms/step - accuracy: 0.9536 - loss: 0.1774 - val_accuracy: 0.3281 - val_loss: 2.4193
Epoch 8/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 5s 167ms/step - accuracy: 0.9914 - loss: 0.0682 - val_accuracy: 0

In [2]:
# Save the trained model
model.save('codebase/men_faceshape_model.h5')

In [3]:
import random
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Load the saved model
model = load_model('codebase/men_faceshape_model.h5')

# Get class labels from the training generator
class_labels = list(train_data.class_indices.keys())

# Pick a random image from the test set
test_dirs = [
    '/Users/rohanrao/Desktop/hair4face/mens faceshape/testing_set/oblong',
    '/Users/rohanrao/Desktop/hair4face/mens faceshape/testing_set/oval',
    '/Users/rohanrao/Desktop/hair4face/mens faceshape/testing_set/round',
    '/Users/rohanrao/Desktop/hair4face/mens faceshape/testing_set/square'
]
test_dir = random.choice(test_dirs)
img_name = random.choice([f for f in os.listdir(test_dir) if f.lower().endswith('.jpg')])
img_path = os.path.join(test_dir, img_name)

# Load and preprocess the image
img = image.load_img(img_path, target_size=IMG_SIZE)
x = image.img_to_array(img)
x = x / 255.0
x = np.expand_dims(x, axis=0)

# Predict
pred = model.predict(x)
pred_class = class_labels[np.argmax(pred)]
print(f"Test image: {img_path}")
print(f"Predicted face shape: {pred_class}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
Test image: /Users/rohanrao/Desktop/hair4face/mens faceshape/testing_set/square/square t82.jpg
Predicted face shape: square
